<a href="https://colab.research.google.com/github/sbbwoy/CodeStudy/blob/main/Dacon_LG_AI_Challenge_for_Plant_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://dacon.io/competitions/official/235870/overview/description

https://github.com/siwooyong/LG-AI-Challenge-for-Plant-Classification

해야할일!
- 이전 코드 다시 돌려보기
- 이번에 사용되는 모르는 함수 공부해오기
- 이번에 사용되는 모델 간략하게 요약하기

#학습 코드

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip uninstall opencv-python-headless==4.5.5.62 --yes
!pip install opencv-python-headless==4.1.2.30

import pickle
import gc
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from tqdm import tqdm
from glob import glob
import random
import os
import json 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split

# timm : 0.5.4
!pip install timm
# albumtations : 1.1.0
!pip install -U albumentations
# transformers : 4.16.2
!pip install transformers

import timm
from albumentations.pytorch.transforms import ToTensorV2
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup

os.environ["CUDA_VISIBLE_DEVICES"]="0"

Found existing installation: opencv-python-headless 4.6.0.66
Uninstalling opencv-python-headless-4.6.0.66:
  Successfully uninstalled opencv-python-headless-4.6.0.66
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.8 MB 4.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 509 kB 4.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.0 MB/s 
     |████████████████████████████████| 120 kB 88.5 MB/s 
     |████████████████████████████████| 6.6 MB 68.6 MB/s 


**timm**
- 사전 학습된 model(e.g. convnext_tiny)들을 pytorch에서 활용할 수 있음

**albumentations**
- image augmentation library
- 다른 image augmentation library들에 비해 빠름
- numpy, OpenCV, imgaug 등 여러 library를 기반으로 optimize

In [32]:
# unzip train data

!unzip "/content/drive/MyDrive/22-여름 스터디/LG_Plant_Classification/data/train.zip"
!unzip "/content/drive/MyDrive/22-여름 스터디/LG_Plant_Classification/data/test.zip"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: test/65823/65823.csv    
  inflating: test/65823/65823.jpg    
   creating: test/65824/
  inflating: test/65824/65824.jpg    
  inflating: test/65824/65824.csv    
   creating: test/65825/
  inflating: test/65825/65825.csv    
  inflating: test/65825/65825.jpg    
   creating: test/65827/
  inflating: test/65827/65827.jpg    
  inflating: test/65827/65827.csv    
   creating: test/65828/
  inflating: test/65828/65828.jpg    
  inflating: test/65828/65828.csv    
   creating: test/65829/
  inflating: test/65829/65829.jpg    
  inflating: test/65829/65829.csv    
   creating: test/65830/
  inflating: test/65830/65830.jpg    
  inflating: test/65830/65830.csv    
   creating: test/65831/
  inflating: test/65831/65831.jpg    
  inflating: test/65831/65831.csv    
   creating: test/65832/
  inflating: test/65832/65832.jpg    
  inflating: test/65832/65832.csv    
   creating: test/65833/
  inflating: test/65833/65833.jpg    
  inflating: test

- pc에서 압축 풀고 구드에 업로드해서 진행

In [33]:
# get sequence feature information

csv_feature_dict = {'내부 습도 1 최고': [25.9, 100.0],
                    '내부 습도 1 최저': [0.0, 100.0],
                    '내부 습도 1 평균': [23.7, 100.0],
                    '내부 온도 1 최고': [3.4, 47.6],
                    '내부 온도 1 최저': [3.3, 47.0],
                    '내부 온도 1 평균': [3.4, 47.3],
                    '내부 이슬점 최고': [0.2, 34.7],
                    '내부 이슬점 최저': [0.0, 34.4],
                    '내부 이슬점 평균': [0.1, 34.5]}

In [34]:
# label encoder, decoder 

train_json = sorted(glob('train/*/*.json'))

labels = []
for i in range(len(train_json)):
    with open(train_json[i], 'r') as f:
        sample = json.load(f)
        crop = sample['annotations']['crop']
        disease = sample['annotations']['disease']
        risk = sample['annotations']['risk']
        label=f"{crop}_{disease}_{risk}"
        labels.append(label)

label_encoder = sorted(np.unique(labels))
label_encoder = {key:value for key,value in zip(label_encoder, range(len(label_encoder)))}
label_decoder = {val:key for key, val in label_encoder.items()}

###JSON
- load() 함수: JSON 파일을 Python 객체로 불러오기
- dump() 함수: Python 객체를 JSON 파일에 저장하기
- loads() 함수: JSON 문자열을 Python 객체로 변환
- dumps() 함수: Python 객체를 JSON 문자열로 변환

In [35]:
# hyper parameters

opt = dict()

opt['batch_size'] = 16
opt['class_n'] = len(label_encoder)
opt['lr'] = 2e-4
opt['embedding_dim'] = 512
opt['feature_n'] = len(csv_feature_dict)
opt['max_len'] = 300
opt['dropout_rate'] = 0.3
opt['epoch_n'] = 25
opt['vision_pretrain'] = True
opt['worker_n'] = 8
opt['folder'] = 'model_weights'
opt['bidirectional'] = True
opt['minmax_dict'] = csv_feature_dict
opt['label_dict'] = label_encoder
opt['enc_name'] = 'resnext50_32x4d'
opt['enc_dim'] = 2048
opt['dec_dim'] = 1024
opt['img_size1'] = 384
opt['img_size2'] = 384
opt['precision'] = 'amp'
opt['seed'] = 42
opt['mix'] = 'cutmix'
opt['mix_prob'] = 0.3
opt['mean'] = [0.485, 0.456, 0.406]
opt['std'] = [0.229, 0.224, 0.225]


device = torch.device("cuda:0")

In [36]:
# fix seed

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(opt['seed'])

In [37]:
# cutmix function

def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2

def cutmix(data, target, alpha):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_target = target[indices]

    lam = np.clip(np.random.beta(alpha, alpha),0.3,0.4)
    bbx1, bby1, bbx2, bby2 = rand_bbox(data.size(), lam)
    new_data = data.clone()
    new_data[:, :, bby1:bby2, bbx1:bbx2] = data[indices, :, bby1:bby2, bbx1:bbx2]
    # adjust lambda to exactly match pixel ratio
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (data.size()[-1] * data.size()[-2]))
    targets = (target, shuffled_target, lam)

    return new_data, targets

In [38]:
train = sorted(glob('train/*'))
test = sorted(glob('test/*'))
labelsss = pd.read_csv("/content/drive/MyDrive/22-여름 스터디/LG_Plant_Classification/data/train.csv")['label']

In [39]:
# k-fold cross validation

skf = StratifiedKFold(n_splits = 5)
train = shuffle(train, random_state=opt['seed'])

folds = []
for idx, (train_idx, val_idx) in enumerate(skf.split(train, labelsss)):
    folds.append((train_idx, val_idx))

**StratifiedKFold**
- target에 피처값의 개수를 동일하게 가져감으로써 kfold 같이 데이터가 한곳으로 몰리는 문제를 방지

In [40]:
# dataset

def trainTransform():
  return Compose([
                  Transpose(p=0.5),
                  HorizontalFlip(p=0.5),
                  VerticalFlip(p=0.5),
                  ShiftScaleRotate(p=0.5),
                  RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
                  Resize(opt['img_size1'], opt['img_size2']),
                  Normalize(mean=opt['mean'], std=opt['std'], max_pixel_value=255.0, p=1.0),
                  ToTensorV2(p=1.0),
                  ], p=1.)
  
def valTransform():
  return Compose([
                  Resize(opt['img_size1'], opt['img_size2']),
                  Normalize(mean=opt['mean'], std=opt['std'], max_pixel_value=255.0, p=1.0),
                  ToTensorV2(p=1.0),
              ], p=1.)

class CustomDataset(Dataset):
    def __init__(self, opt, files, mode, transforms):
        self.files = files
        self.mode = mode
        self.transforms = transforms
        self.csv_check = [0]*len(self.files)
        self.seq = [None]*len(self.files)
        self.minmax_dict = opt['minmax_dict']
        self.max_len = opt['max_len']
        self.label_encoder = opt['label_dict']

    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        file = self.files[i]
        file_name = file.split('/')[-1]
        
        if self.csv_check[i] == 0:
            csv_path = f'{file}/{file_name}.csv'
            df = pd.read_csv(csv_path)
            try:
                estiTime1, estiTime2 = df.iloc[0]['측정시각'], df.iloc[1]['측정시각']
            except:
                estiTime1, estiTime2 = 0, 1

            df = df[self.minmax_dict.keys()]
            df = df.replace('-', 0)
            
            if self.mode == 'train':
                decision = np.random.rand()
                if estiTime1==estiTime2 and len(df)>400:
                    if decision > 0.5:
                        df = df[0::2].reset_index(drop=True)
                    else:
                        df = df[1::2].reset_index(drop=True)
            else: 
                if estiTime1==estiTime2 and len(df)>400:
                    df = df[0::2].reset_index(drop=True)
                
            
            # minmax-scaling
            for col in df.columns:
                df[col] = df[col].astype(float) - self.minmax_dict[col][0]
                df[col] = df[col] / (self.minmax_dict[col][1]-self.minmax_dict[col][0])

            # zero-padding
            pad = np.zeros((self.max_len, len(df.columns)))
            length = min(self.max_len, len(df))
            pad[-length:] = df.to_numpy()[-length:]

            # transpose-to-sequential-data
            seq = torch.tensor(pad, dtype=torch.float32)
            self.seq[i] = seq
            self.csv_check[i] = 1
        else:
            seq = self.seq[i]
        
        # image-transform
        image_path = f'{file}/{file_name}.jpg'
        img = cv2.imread(image_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.uint8)              
        img = self.transforms(image=img)['image'] 

        
        if self.mode == 'train' or self.mode == 'val':
            json_path = f'{file}/{file_name}.json'
            with open(json_path, 'r') as f:
                json_file = json.load(f)
            
            crop = json_file['annotations']['crop']
            disease = json_file['annotations']['disease']
            risk = json_file['annotations']['risk']
            label = torch.tensor(self.label_encoder[f'{crop}_{disease}_{risk}'], dtype=torch.long)
            
            return img, seq, label
        else:
            return img, seq

In [41]:
# model

class Encoder(nn.Module):
    def __init__(self, opt):
        super(Encoder, self).__init__()
        self.model = timm.create_model(model_name=opt['enc_name'], 
                                       pretrained=opt['vision_pretrain'], 
                                       num_classes=0)
    
    def forward(self, inputs):
        output = self.model(inputs)
        return output


class Decoder(nn.Module):
    def __init__(self, opt):
        super(Decoder, self).__init__()
        self.decoder = nn.GRU(opt['feature_n'], opt['embedding_dim'], bidirectional = opt['bidirectional'])
        self.dense = nn.Linear(2*opt['max_len']*opt['embedding_dim'], opt['dec_dim'])
        
        self.f1 = nn.Linear(opt['enc_dim']+opt['dec_dim'], opt['enc_dim']+opt['dec_dim'])
        self.out = nn.Linear(opt['enc_dim']+opt['dec_dim'], opt['class_n'])
        self.dropout = nn.Dropout(opt['dropout_rate'])
        self.relu = nn.ReLU()

    def init_weight(self):
        torch.nn.init.xavier_uniform_(self.f1.weight)  
        torch.nn.init.xavier_uniform_(self.dense.weight)  
        torch.nn.init.xavier_uniform_(self.out.weight)  


    def forward(self, enc_out, dec_inp):
        dec_out, _ = self.decoder(dec_inp)
        dec_out = self.dense(dec_out.view(dec_out.size(0), -1))

        concat = torch.cat([enc_out, dec_out], dim=1) 
        concat = self.f1(self.relu(concat))
        concat = self.dropout(self.relu(concat))
        output = self.out(concat)
        return output


class CustomModel(nn.Module):
    def __init__(self, opt):
        super(CustomModel, self).__init__()
        self.encoder = Encoder(opt)
        self.decoder = Decoder(opt)
        self.to(device)
        
    def forward(self, img, seq):
        enc_out = self.encoder(img)
        output = self.decoder(enc_out, seq)
        
        return output

In [42]:
# train function

class CustomTrainer:
    def __init__(self, model, folder, fold):
        self.model=model
        
        self.save_dir = f'/content/{folder}'
        if not os.path.exists(self.save_dir):
          os.makedirs(self.save_dir)
          
        self.optimizer = AdamW(model.parameters(), lr=opt['lr'])
        self.scaler = torch.cuda.amp.GradScaler() 

        total_steps = int(len(train_dataset)*opt['epoch_n']/(opt['batch_size']))
        warmup_steps = 1149
        print('total_steps: ', total_steps)
        print('warmup_steps: ', warmup_steps)
        self.scheduler = get_cosine_schedule_with_warmup(self.optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)
        self.loss_fn = nn.CrossEntropyLoss()
        self.val_loss_fn = nn.CrossEntropyLoss()

        self.best_score = 0.0


    def run(self, train_loader, val_loader):
        for epoch in range(opt['epoch_n']):
            gc.collect()
            learning_rate = self.optimizer.param_groups[0]['lr']
            print('learning_rate: ', learning_rate)
            print(f'----- train, epoch{epoch+1} -----')
            train_loss, train_score = self.train_function(train_loader, epoch)
            print(' ')
            print(f'train_loss: {train_loss:.6f}, train_score: {train_score:.6f}')

            print('----------------------------------')

            print(f'----- val, epoch{epoch+1} -----')
            with torch.no_grad():
                val_loss, val_score = self.val_function(val_loader)
            print(' ')
            print(f'val_loss: {val_loss:.6f}, val_score: {val_score:.6f}')


            if epoch+1 >= 16 and val_score >= self.best_score:
                torch.save(self.model.state_dict(), self.save_dir+f"/best-acc-epoch{epoch+1}.bin")
                self.best_score=val_score
                print(f'model is saved when epoch is : {epoch+1}')


    def train_function(self, train_loader, epoch):
        self.model.train()

        total_loss = 0.0
        total_score = 0.0
        for bi, data in enumerate(tqdm(train_loader)):
            data = [x.to(device) for x in data]
            img, seq, label = data

            self.optimizer.zero_grad()
            
            # use mix or not
            if opt['mix']!=None and epoch < opt['epoch_n']-10: 
                mix_decision = np.random.rand()
                if opt['mix'] == 'cutmix' and mix_decision < opt['mix_prob']:
                    img, mix_labels = cutmix(img, label, 1.0)
                else: 
                  pass
            else: mix_decision = 1

            if opt['epoch_n']-10 <= epoch:
                assert mix_decision == 1
            
            # use amp or not
            if opt['precision'] == 'float':
                out = self.model(img, seq)

                if mix_decision < opt['mix_prob']:
                    loss = self.loss_fn(out, mix_labels[0])*mix_labels[2] + self.loss_fn(out, mix_labels[1])*(1-mix_labels[2])
                else:
                    loss = self.loss_fn(out, label)

                loss.backward()
                self.optimizer.step()
            else: 
                with torch.cuda.amp.autocast():
                    out = self.model(img, seq)
                    if mix_decision < opt['mix_prob']:
                        loss = self.loss_fn(out, mix_labels[0])*mix_labels[2] + self.loss_fn(out, mix_labels[1])*(1-mix_labels[2])
                    else:
                        loss = self.loss_fn(out, label)

                self.scaler.scale(loss).backward()  
                self.scaler.step(self.optimizer) 
                self.scaler.update()              
            
            self.scheduler.step()
            total_loss+=loss.detach().cpu()

            total_score+=f1_score(label.cpu(), out.argmax(1).cpu(), average='macro')
        return total_loss/len(train_loader), total_score/len(train_loader)

    def val_function(self, val_loader):
        self.model.eval()

        total_loss = 0.0
        preds, targets = [], []
        for bi, data in enumerate(tqdm(val_loader)):
            data = [x.to(device) for x in data]
            img, seq, label = data

            out = self.model(img, seq)
            loss = self.val_loss_fn(out, label)

            total_loss+=loss.detach().cpu()

            pred = out.argmax(1).detach().cpu().tolist()
            target = label.reshape(-1).detach().cpu().tolist()

            preds.extend(pred)
            targets.extend(target)
        
        score = f1_score(targets, preds, average='macro')
        return total_loss/len(val_loader), score

    def log(self, message):
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')

In [ ]:
# run

print(opt)
for i in range(len(folds)):
    train_idx, val_idx = folds[i]
    
    print(f'{i+1}th fold training is start')
    
    # data
    train_dataset = CustomDataset(opt, np.array(train)[train_idx], mode='train', transforms=trainTransform())
    val_dataset = CustomDataset(opt, np.array(train)[val_idx], mode='val', transforms=valTransform())
    print('num of train: ', len(train_dataset))
    print('num of val: ', len(val_dataset))

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=opt['batch_size'], num_workers=opt['worker_n'], shuffle=True, drop_last=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2*opt['batch_size'], num_workers=opt['worker_n'], shuffle=False)

    # model
    custom_model = CustomModel(opt)

    # trainer
    custom_trainer = CustomTrainer(model=custom_model, folder=opt['folder']+f'/fold{i+1}', fold=i+1)
    custom_trainer.run(train_dataloader, val_dataloader)

{'batch_size': 16, 'class_n': 25, 'lr': 0.0002, 'embedding_dim': 512, 'feature_n': 9, 'max_len': 300, 'dropout_rate': 0.3, 'epoch_n': 25, 'vision_pretrain': True, 'worker_n': 8, 'folder': 'model_weights', 'bidirectional': True, 'minmax_dict': {'내부 습도 1 최고': [25.9, 100.0], '내부 습도 1 최저': [0.0, 100.0], '내부 습도 1 평균': [23.7, 100.0], '내부 온도 1 최고': [3.4, 47.6], '내부 온도 1 최저': [3.3, 47.0], '내부 온도 1 평균': [3.4, 47.3], '내부 이슬점 최고': [0.2, 34.7], '내부 이슬점 최저': [0.0, 34.4], '내부 이슬점 평균': [0.1, 34.5]}, 'label_dict': {'1_00_0': 0, '2_00_0': 1, '2_a5_2': 2, '3_00_0': 3, '3_a9_1': 4, '3_a9_2': 5, '3_a9_3': 6, '3_b3_1': 7, '3_b6_1': 8, '3_b7_1': 9, '3_b8_1': 10, '4_00_0': 11, '5_00_0': 12, '5_a7_2': 13, '5_b6_1': 14, '5_b7_1': 15, '5_b8_1': 16, '6_00_0': 17, '6_a11_1': 18, '6_a11_2': 19, '6_a12_1': 20, '6_a12_2': 21, '6_b4_1': 22, '6_b4_3': 23, '6_b5_1': 24}, 'enc_name': 'resnext50_32x4d', 'enc_dim': 2048, 'dec_dim': 1024, 'img_size1': 384, 'img_size2': 384, 'precision': 'amp', 'seed': 42, 'mix': 'cutmix', 

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


total_steps:  7207
warmup_steps:  1149
learning_rate:  0.0
----- train, epoch1 -----


  1%|▏         | 4/288 [00:16<11:13,  2.37s/it]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check 

 
train_loss: 2.341666, train_score: 0.164272
----------------------------------
----- val, epoch1 -----


...

  0%|          | 0/288 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 288/288 [02:04<00:00,  2.31it/s]


 
train_loss: 0.006144, train_score: 0.995190
----------------------------------
----- val, epoch21 -----


  0%|          | 0/37 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 37/37 [00:18<00:00,  2.00it/s]


 
val_loss: 0.103198, val_score: 0.927764
learning_rate:  1.7525257902635262e-05
----- train, epoch22 -----


  0%|          | 0/288 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
 20%|██        | 59/288 [00:27<01:38,  2.33it/s]

#추론 코드

In [ ]:
!pip uninstall opencv-python-headless==4.5.5.62 --yes
!pip install opencv-python-headless==4.1.2.30

import pickle
import gc
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from glob import glob
from sklearn.utils import shuffle
import random
import os
import json 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import models
from torch.utils.data import Dataset
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split

!pip install timm
!pip install -U albumentations 

import timm
from albumentations.pytorch.transforms import ToTensorV2
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
# get sequence feature information

csv_feature_dict = {'내부 습도 1 최고': [25.9, 100.0],
                    '내부 습도 1 최저': [0.0, 100.0],
                    '내부 습도 1 평균': [23.7, 100.0],
                    '내부 온도 1 최고': [3.4, 47.6],
                    '내부 온도 1 최저': [3.3, 47.0],
                    '내부 온도 1 평균': [3.4, 47.3],
                    '내부 이슬점 최고': [0.2, 34.7],
                    '내부 이슬점 최저': [0.0, 34.4],
                    '내부 이슬점 평균': [0.1, 34.5]}

In [ ]:
# label encoder, decoder 

train_json = sorted(glob('train/*/*.json'))

labels = []
for i in range(len(train_json)):
    with open(train_json[i], 'r') as f:
        sample = json.load(f)
        crop = sample['annotations']['crop']
        disease = sample['annotations']['disease']
        risk = sample['annotations']['risk']
        label=f"{crop}_{disease}_{risk}"
        labels.append(label)

label_encoder = sorted(np.unique(labels))
label_encoder = {key:value for key,value in zip(label_encoder, range(len(label_encoder)))}
label_decoder = {val:key for key, val in label_encoder.items()}

In [ ]:
# hyper parameters

opt = dict()

opt['batch_size'] = 16
opt['class_n'] = len(label_encoder)
opt['lr'] = 2e-4
opt['embedding_dim'] = 512
opt['feature_n'] = len(csv_feature_dict)
opt['max_len'] = 300
opt['dropout_rate'] = 0.3
opt['epoch_n'] = 25
opt['vision_pretrain'] = True
opt['worker_n'] = 8
opt['folder'] ='model_weights'
opt['bidirectional'] = True
opt['minmax_dict'] = csv_feature_dict
opt['label_dict'] = label_encoder
opt['enc_name'] = 'resnext50_32x4d'
opt['enc_dim'] = 2048
opt['dec_dim'] = 1024
opt['img_size1'] = 384
opt['img_size2'] = 384
opt['precision'] = 'amp'
opt['seed'] = 42
opt['mix'] = 'cutmix'
opt['mix_prob'] = 0.3
opt['mean'] = [0.485, 0.456, 0.406]
opt['std'] = [0.229, 0.224, 0.225]


device = torch.device("cuda:0")

In [ ]:
# fix seed

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(opt['seed'])

In [ ]:
# get data

test = sorted(glob('test/*'))

In [ ]:
# model

class Encoder(nn.Module):
    def __init__(self, opt):
        super(Encoder, self).__init__()
        self.model = timm.create_model(model_name=opt['enc_name'], 
                                       pretrained=opt['vision_pretrain'], 
                                       num_classes=0)
    
    def forward(self, inputs):
        output = self.model(inputs)
        return output


class Decoder(nn.Module):
    def __init__(self, opt):
        super(Decoder, self).__init__()
        self.decoder = nn.GRU(opt['feature_n'], opt['embedding_dim'], bidirectional = opt['bidirectional'])
        self.dense = nn.Linear(2*opt['max_len']*opt['embedding_dim'], opt['dec_dim'])
        
        self.f1 = nn.Linear(opt['enc_dim']+opt['dec_dim'], opt['enc_dim']+opt['dec_dim'])
        self.out = nn.Linear(opt['enc_dim']+opt['dec_dim'], opt['class_n'])
        self.dropout = nn.Dropout(opt['dropout_rate'])
        self.relu = nn.ReLU()

    def init_weight(self):
        torch.nn.init.xavier_uniform_(self.f1.weight)  
        torch.nn.init.xavier_uniform_(self.dense.weight)  
        torch.nn.init.xavier_uniform_(self.out.weight)  


    def forward(self, enc_out, dec_inp):
        dec_out, _ = self.decoder(dec_inp)
        dec_out = self.dense(dec_out.view(dec_out.size(0), -1))

        concat = torch.cat([enc_out, dec_out], dim=1) 
        concat = self.f1(self.relu(concat))
        concat = self.dropout(self.relu(concat))
        output = self.out(concat)
        return output


class CustomModel(nn.Module):
    def __init__(self, opt):
        super(CustomModel, self).__init__()
        self.encoder = Encoder(opt)
        self.decoder = Decoder(opt)
        self.to(device)
        
    def forward(self, img, seq):
        enc_out = self.encoder(img)
        output = self.decoder(enc_out, seq)
        
        return output

In [ ]:
# get model weights

custom_model = CustomModel(opt)

model1_path = glob(opt['folder'] + '/fold1/*.bin')[-1]
model2_path = glob(opt['folder'] + '/fold2/*.bin')[-1]
model3_path = glob(opt['folder'] + '/fold3/*.bin')[-1]
model4_path = glob(opt['folder'] + '/fold4/*.bin')[-1]
model5_path = glob(opt['folder'] + '/fold5/*.bin')[-1]

# fold1 model
model1 = CustomModel(opt)
model1.load_state_dict(torch.load(model1_path, map_location='cpu'))
model1.to(device)
model1.eval()

# fold2 model
model2 = CustomModel(opt)
model2.load_state_dict(torch.load(model2_path, map_location='cpu'))
model2.to(device)
model2.eval()

# fold3 model
model3 = CustomModel(opt)
model3.load_state_dict(torch.load(model3_path, map_location='cpu'))
model3.to(device)
model3.eval()

# fold4 model
model4 = CustomModel(opt)
model4.load_state_dict(torch.load(model4_path, map_location='cpu'))
model4.to(device)
model4.eval()

# fold5 model
model5 = CustomModel(opt)
model5.load_state_dict(torch.load(model5_path, map_location='cpu'))
model5.to(device)
model5.eval()

'model weight is loaded'

In [ ]:
# inference dataset

def valTransform():
  return Compose([
                  Resize(opt['img_size1'], opt['img_size2']),
                  Normalize(mean=opt['mean'], std=opt['std'], max_pixel_value=255.0, p=1.0),
                  ToTensorV2(p=1.0),
              ], p=1.)
  
class InferenceDataset(Dataset):
    def __init__(self, opt, files, mode):
        self.files = files
        self.mode = mode
        self.csv_check = [0]*len(self.files)
        self.seq = [None]*len(self.files)
        self.minmax_dict = opt['minmax_dict']
        self.max_len = opt['max_len']
        self.label_encoder = opt['label_dict']

    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        file = self.files[i]
        file_name = file.split('/')[-1]
        
        if self.csv_check[i] == 0:
            csv_path = f'{file}/{file_name}.csv'
            df = pd.read_csv(csv_path)
            try:
                estiTime1, estiTime2 = df.iloc[0]['측정시각'], df.iloc[1]['측정시각']
            except:
                estiTime1, estiTime2 = 0, 1

            df = df[self.minmax_dict.keys()]
            df = df.replace('-', 0)
            
            if estiTime1==estiTime2 and len(df)>400:
                df = df[0::2].reset_index(drop=True)
                
            
            # minmax-scaling
            for col in df.columns:
                df[col] = df[col].astype(float) - self.minmax_dict[col][0]
                df[col] = df[col] / (self.minmax_dict[col][1]-self.minmax_dict[col][0])

            # zero-padding
            pad = np.zeros((self.max_len, len(df.columns)))
            length = min(self.max_len, len(df))
            pad[-length:] = df.to_numpy()[-length:]

            # transpose-to-sequential-data
            seq = torch.tensor(pad, dtype=torch.float32)
            self.seq[i] = seq
            self.csv_check[i] = 1
        else:
            seq = self.seq[i]
        
        image_path = f'{file}/{file_name}.jpg'
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.uint8)
        img = valTransform()(image=img)['image'] 

        if self.mode == 'val':
            json_path = f'{file}/{file_name}.json'
            with open(json_path, 'r') as f:
                json_file = json.load(f)
            
            crop = json_file['annotations']['crop']
            disease = json_file['annotations']['disease']
            risk = json_file['annotations']['risk']
            label = torch.tensor(self.label_encoder[f'{crop}_{disease}_{risk}'], dtype=torch.long)
            
            return img, seq, label
        else:
            return img, seq

In [ ]:
# predict function

def predict(models, loader, mode):
    model1, model2, model3, model4, model5 = models

    preds = []
    for bi, data in enumerate(tqdm(loader)):
        data = [x.to(device) for x in data]
        if mode=='val':
            img, seq, label = data
        else:
            img, seq = data
        output1 = nn.Softmax(dim=-1)(model1(img, seq))
        output2 = nn.Softmax(dim=-1)(model2(img, seq))
        output3 = nn.Softmax(dim=-1)(model3(img, seq))
        output4 = nn.Softmax(dim=-1)(model4(img, seq))
        output5 = nn.Softmax(dim=-1)(model5(img, seq))

        output = output1 + output2 + output3 + output4 + output5
        pred = torch.argmax(output, dim=1).cpu().tolist()
        preds.extend(pred)
    return preds

In [ ]:
# get test preds

models = [model1, model2, model3, model4, model5]

test_dataset = InferenceDataset(opt, test, mode='test')
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=2*opt['batch_size'], num_workers=opt['worker_n'], shuffle=False)

with torch.no_grad():
    preds = predict(models, test_dataloader, mode='test')

In [ ]:
preds = np.array([label_decoder[int(x)] for x in preds])

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/22-여름 스터디/LG_Plant_Classification/data/sample_submission.csv')
submission['label'] = preds

In [ ]:
submission.to_csv('cnn2rnn(resnext-5fold-ensemble)_submission.csv', index=False)